# 第4章: 形態素解析

夏目漱石の小説『吾輩は猫である』の文章（[neko.txt](https://nlp100.github.io/data/neko.txt)）をMeCabを使って形態素解析し，その結果をneko.txt.mecabというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

なお，問題37, 38, 39は[matplotlib](http://matplotlib.org/)もしくは[Gnuplot](http://www.gnuplot.info/)を用いるとよい．


In [1]:
!wget https://nlp100.github.io/data/neko.txt

--2022-12-04 20:39:20--  https://nlp100.github.io/data/neko.txt
Resolving nlp100.github.io (nlp100.github.io)... 185.199.111.153, 185.199.109.153, 185.199.108.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 975789 (953K) [text/plain]
Saving to: ‘neko.txt.1’

neko.txt.1          100%[===================>] 952.92K  --.-KB/s    in 0.03s   

2022-12-04 20:39:20 (28.0 MB/s) - ‘neko.txt.1’ saved [975789/975789]



In [23]:
!sudo apt install mecab libmecab-dev mecab-ipadic-utf8
!mecab neko.txt -o neko.txt.mecab

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


## 30. 形態素解析結果の読み込み
---
形態素解析結果（neko.txt.mecab）を読み込むプログラムを実装せよ．ただし，各形態素は表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をキーとするマッピング型に格納し，1文を形態素（マッピング型）のリストとして表現せよ．第4章の残りの問題では，ここで作ったプログラムを活用せよ．

In [39]:
sentences = []
sentence = []
with open("./neko.txt.mecab") as f:
    for line in f:
        if line == "EOS\n":
            sentences.append(sentence)
            sentence = []
        else:
            surface, feature = line.split("\t")
            features = feature.split(",")
            morpheme_dict = {
                "surface": surface,
                "base": features[6],
                "pos": features[0],
                "pos1": features[1],
            }
            sentence.append(morpheme_dict)

for sentence in sentences:
    print(sentences)

[{'surface': '一', 'base': '一', 'pos': '名詞', 'pos1': '数'}, {'surface': '', 'base': '*\n', 'pos': '記号', 'pos1': '一般'}]
[{'surface': '', 'base': '*\n', 'pos': '記号', 'pos1': '一般'}]
[{'surface': '\u3000', 'base': '\u3000', 'pos': '記号', 'pos1': '空白'}, {'surface': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'}, {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}, {'surface': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'}, {'surface': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'}, {'surface': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'}, {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}, {'surface': '', 'base': '*\n', 'pos': '記号', 'pos1': '一般'}]
[{'surface': '名前', 'base': '名前', 'pos': '名詞', 'pos1': '一般'}, {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}, {'surface': 'まだ', 'base': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続'}, {'surface': '無い', 'base': '無い', 'pos': '形容詞', 'pos1': '自立'}, {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}, {'surface': '', 'base': '*\n', 'pos': '

## 31. 動詞Permalink
---
動詞の表層形をすべて抽出せよ．

## 32. 動詞の基本形
---
動詞の基本形をすべて抽出せよ．

## 33. 「AのB」
---
2つの名詞が「の」で連結されている名詞句を抽出せよ．

## 34. 名詞の連接
---
名詞の連接（連続して出現する名詞）を最長一致で抽出せよ．

## 35. 単語の出現頻度
---
文章中に出現する単語とその出現頻度を求め，出現頻度の高い順に並べよ．

## 36. 頻度上位10語
---
出現頻度が高い10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

## 37. 「猫」と共起頻度の高い上位10語
---
「猫」とよく共起する（共起頻度が高い）10語とその出現頻度をグラフ（例えば棒グラフなど）で表示せよ．

## 38. ヒストグラム
---
単語の出現頻度のヒストグラムを描け．ただし，横軸は出現頻度を表し，1から単語の出現頻度の最大値までの線形目盛とする．縦軸はx軸で示される出現頻度となった単語の異なり数（種類数）である．

## 39. Zipfの法則
---
単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．